In [ ]:
from pathlib import Path
from copy import deepcopy
import json
import subprocess
import sys
spec_home="/home/hchaudha/spec"

In [ ]:
def path_list_to_str_list(data):
  copy_data = deepcopy(data)
  for i,val in enumerate(copy_data):
    copy_data[i] = str(copy_data[i])
  return copy_data

def path_dict_to_str_dict(data):
  copy_data = deepcopy(data)
  for key in copy_data:
    if isinstance(copy_data[key],dict):
      copy_data[key] = path_dict_to_str_dict(copy_data[key])
    if isinstance(copy_data[key],Path):
      copy_data[key] = str(copy_data[key])
    if isinstance(copy_data[key],list):
      copy_data[key] = path_list_to_str_list(copy_data[key])

  return copy_data

def sort_paths_by_postfix(paths):
  # Define a helper function to extract the postfix for sorting
  def extract_postfix(path):
      # Split the path by '/' and get the last segment
      # Then get the part after the last underscore '_'
      if isinstance(path,Path):
        return str(path).split('/')[-1].split('_')[-1]
      else:
        return path.split('/')[-1].split('_')[-1]

  # Sort the paths using the postfix as key
  sorted_paths = sorted(paths, key=extract_postfix)
  return sorted_paths

In [ ]:
lev_paths_dict = {}
lev_paths_dict["high_accuracy_L02_master"] = {
  "Ev_path": "/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev",
  "save_path": "/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data",
  "Lev0":[ ],
  "Lev1":[ ],
  "Lev2":[ ],
}

In [ ]:
lev_paths_dict
with open("./del.json",'w') as f:
  json.dump(lev_paths_dict,f,indent=2)

In [ ]:
run_name = "Lev5_big_gaussian"
Ev_path = Path(f"/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/{run_name}")
save_path = Ev_path/"GW_data"

lev_paths_dict[run_name] = {
  "Ev_path":Ev_path,
  "save_path":save_path,
}

lev_paths = list(Ev_path.glob("Lev?_??"))
levs = set()
for paths in lev_paths:
  levs.add(str(paths.stem)[3])

for lev in levs:
  lev_paths = []
  lev_paths = lev_paths + list(Ev_path.glob(f"Lev{lev}_??"))
  lev_paths = lev_paths + list(Ev_path.glob(f"Lev{lev}_Ringdown/Lev{lev}_??"))
  lev_paths.sort()
  lev_paths_dict[run_name][f"Lev{lev}"] = path_list_to_str_list(lev_paths)

# save_path.mkdir(parents=True, exist_ok=False)

with (save_path/"paths.json").open('w') as f:
  json.dump(path_dict_to_str_dict(lev_paths_dict),f,indent=2)

## Write paths of levs

In [ ]:
def submit_joinH5_command(save_folder:Path, path_list:list[Path], output_file_name:str, write_scripts_only=False):
  file_list_str = ""
  for file_path in path_list:
    file_list_str += f" {file_path}"

  command = f"cd {save_folder} && {spec_home}/Support/bin/JoinH5 -o {output_file_name} {file_list_str}"
  with open(save_folder/"joinH5_commands.sh",'w') as f:
    submit_file =\
f"""#!/bin/bash -
# run JoinH5
{command}
"""
    f.writelines(submit_file)

  if not write_scripts_only:
    status = subprocess.run(command, capture_output=True, shell=True, text=True)
    if status.returncode == 0:
      print(f"Succesfully saved joined h5 file {output_file_name} in {save_folder}")
    else:
      sys.exit(
          f"JoinH5 failed in {save_folder} with error: \n {status.stderr}")

def get_GW_data_for_levs(lev_path_list:list[Path]):
  # NOTE! No sorting is done in this function. It is important that the lev_path_list is in the correct order which will be passed to the JoinH5
  GW_data = dict()
  bondi_radii_list = set()
  # This uses the first path to get the list of the radii, which could be different from the list of radii in the subsequent folders in case fo checkpoints restarts
  for bondi_path in list(lev_path_list[0].glob("Run/GW2/Bondi*.h5")):
    bondi_radii_list.add(bondi_path.stem[-4:])

  GW_data[f"bondi_radii_list"] = list(bondi_radii_list)
  GW_data[f"PhiMinus_FiniteRadii_CodeUnits"] = []
  GW_data[f"PhiPlus_FiniteRadii_CodeUnits"] = []
  GW_data[f"RawStrahlkorperIntegrals"] = []
  GW_data[f"rh_FiniteRadii_CodeUnits"] = []
  GW_data[f"rPsi4_FiniteRadii_CodeUnits"] = []
  for bondi_radii in bondi_radii_list:
    GW_data[f"BondiCceR{bondi_radii}"] = []

  def append_if_exists(path_list:list[Path],path:Path):
    if path.exists():
      path_list.append(path)
    else:
      print(f"{path=} does not exist. Skipping it!")

  for lev_path in lev_path_list:
    append_if_exists(GW_data[f"PhiMinus_FiniteRadii_CodeUnits"],Path(f"{lev_path}/Run/GW2/PhiMinus_FiniteRadii_CodeUnits.h5"))
    append_if_exists(GW_data[f"PhiPlus_FiniteRadii_CodeUnits"],Path(f"{lev_path}/Run/GW2/PhiPlus_FiniteRadii_CodeUnits.h5"))
    append_if_exists(GW_data[f"RawStrahlkorperIntegrals"],Path(f"{lev_path}/Run/GW2/RawStrahlkorperIntegrals.h5"))
    append_if_exists(GW_data[f"rh_FiniteRadii_CodeUnits"],Path(f"{lev_path}/Run/GW2/rh_FiniteRadii_CodeUnits.h5"))
    append_if_exists(GW_data[f"rPsi4_FiniteRadii_CodeUnits"],Path(f"{lev_path}/Run/GW2/rPsi4_FiniteRadii_CodeUnits.h5"))
    for bondi_radii in bondi_radii_list:
      append_if_exists(GW_data[f"BondiCceR{bondi_radii}"],Path(f"{lev_path}/Run/GW2/BondiCceR{bondi_radii}.h5"))

  return GW_data

def make_config_file(BoundaryDataPath: Path,
                     InputSavePath: Path = None) -> Path :
        
    if InputSavePath is None:
        InputSavePath = BoundaryDataPath.parent/"cce.yaml"
    assert(InputSavePath.parent.exists())

    config_file=\
f"""
# Distributed under the MIT License.
# See LICENSE.txt for details.

# This block is used by testing and the SpECTRE command line interface.
Executable: CharacteristicExtract
Testing:
  Check: parse
  Priority: High

---
Evolution:
  InitialTimeStep: 0.25
  InitialSlabSize: 10.0

ResourceInfo:
  AvoidGlobalProc0: false
  Singletons: Auto

Observers:
  VolumeFileName: "vol_{str(InputSavePath.stem)}"
  ReductionFileName: "red_{str(InputSavePath.stem)}"

EventsAndTriggers:
  # Write the CCE time step every Slab. A Slab is a fixed length of simulation
  # time and is not influenced by the dynamically adjusted step size.
  - Trigger:
      Slabs:
        EvenlySpaced:
          Offset: 0
          Interval: 1
    Events:
      - ObserveTimeStep:
          # The output is written into the "ReductionFileName" HDF5 file under
          # "/SubfileName.dat"
          SubfileName: CceTimeStep
          PrintTimeToTerminal: true

Cce:
  Evolution:
    TimeStepper:
      AdamsBashforth:
        Order: 3 # Going to higher order doesn't seem necessary for CCE
    StepChoosers:
      - Constant: 0.1 # Don't take steps bigger than 0.1M
      - Increase:
          Factor: 2
      - ErrorControl(SwshVars):
          AbsoluteTolerance: 1e-8
          RelativeTolerance: 1e-6
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9
      - ErrorControl(CoordVars):
          AbsoluteTolerance: 1e-8
          RelativeTolerance: 1e-7
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9

  # The number of angular modes used by the CCE evolution. This must be larger
  # than ObservationLMax. We always use all of the m modes for the LMax since
  # using fewer m modes causes aliasing-driven instabilities.
  LMax: 20
  # Probably don't need more than 15 radial grid points, but could increase
  # up to ~20
  NumberOfRadialPoints: 15
  # The maximum ell we use for writing waveform output. While CCE can dump
  # more, you should be cautious with higher modes since mode mixing, truncation
  # error, and systematic numerical effects can have significant contamination
  # in these modes.
  ObservationLMax: 8

  InitializeJ:
    # To see what other J-initialization procedures are available, comment
    # out this group of options and do, e.g. "Blah:" The code will print
    # an error message with the available options and a help string.
    # More details can be found at spectre-code.org.
    ConformalFactor:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000 # Do extra iterations in case we improve.
      RequireConvergence: False # Often don't converge to 1e-13, but that's fine
      OptimizeL0Mode: True
      UseBetaIntegralEstimate: False
      ConformalFactorIterationHeuristic: SpinWeight1CoordPerturbation
      UseInputModes: False
      InputModes: []

  StartTime: Auto
  EndTime: Auto
  ExtractionRadius: Auto

  BoundaryDataFilename: {BoundaryDataPath.name}
  H5IsBondiData: True
  H5Interpolator:
    BarycentricRationalSpanInterpolator:
      MinOrder: 10
      MaxOrder: 10
  FixSpecNormalization: False

  H5LookaheadTimes: 10000

  Filtering:
    RadialFilterHalfPower: 64
    RadialFilterAlpha: 35.0
    FilterLMax: 18

  ScriInterpOrder: 5
  ScriOutputDensity: 1

"""

    with InputSavePath.open('w') as f:
        f.writelines(config_file)

    return InputSavePath

def make_submit_file(save_folder_path:Path,cce_input_file_path:Path, CCE_Executable_path:Path,write_scripts_only=False):
  submit_script=\
f"""#!/bin/bash -
#SBATCH -J CCE_{save_folder_path.stem}             # Job Name
#SBATCH -o CCE.stdout                 # Output file name
#SBATCH -e CCE.stderr                 # Error file name
#SBATCH -n 4                          # Number of cores
#SBATCH -p expansion                  # Queue name
#SBATCH --ntasks-per-node 4           # number of MPI ranks per node
#SBATCH -t 24:0:00   # Run time
#SBATCH -A sxs                # Account name
#SBATCH --no-requeue
#SBATCH --reservation=sxs_standing
#SBATCH --constraint=skylake

# Go to the correct folder with the boundary data
cd {save_folder_path}

# run CCE
{CCE_Executable_path} --input-file ./{cce_input_file_path.name}
"""
  submit_script_path = save_folder_path/"submit.sh"
  submit_script_path.write_text(submit_script)

  if not write_scripts_only:
    command = f"cd {save_folder_path} && qsub {submit_script_path}"
    status = subprocess.run(command, capture_output=True, shell=True, text=True)
    if status.returncode == 0:
      print(f"Succesfully submitted {submit_script_path}\n{status.stdout}")
    else:
      sys.exit(
          f"Job submission failed for {submit_script_path} with error: \n{status.stdout} \n{status.stderr}")

def serialize_and_dump_dict(data_dict:dict, save_path:Path):
  def serialize(data):
      """Recursively serialize Path objects and convert sets to lists."""
      if isinstance(data, dict):
          return {key: serialize(value) for key, value in data.items()}
      elif isinstance(data, list):
          return [serialize(item) for item in data]
      elif isinstance(data, set):
          return list(serialize(item) for item in data)  # Convert set to list
      elif isinstance(data, Path):
          return str(data)  # Convert Path to string
      return data  # Return unmodified if it's not a Path, dict, list, or set

  # Serialize the data
  serialized_data = serialize(data_dict)

  # Write the serialized data to a JSON file
  with open(save_path, 'w') as json_file:
      json.dump(serialized_data, json_file, indent=2)

def write_GW_data_dict(Ev_path:Path, CCE_Executable_path:Path, Ev_lev_path_glob:str=None, save_path:Path=None, previous_levs_list:list[Path]=None,write_scripts_only:bool=False):

  if save_path is None:
    save_path = Ev_path/"GW_data"
  # Create save folder
  save_path.mkdir()

  if previous_levs_list is None:
    has_previous_levs = False
    previous_levs_list = []
  else:
  # If there are previous levs then check that the paths are correct
    has_previous_levs = True
    for previous_lev_path in previous_levs_list:
      if not previous_lev_path.exists():
        raise Exception(f"{previous_lev_path=} does not exist!")

  GW_data_dict= {
    "Ev_path":Ev_path,
    "Ev_lev_path_glob":Ev_lev_path_glob,
    "save_path":save_path,
    "has_previous_levs": has_previous_levs,
  }

  # Find all the levs in the Ev folder, if Ev_lev_path_glob is not None then use it for globbing
  if Ev_lev_path_glob is None:
    Ev_lev_path_glob = "Lev?_??"

  Ev_lev_paths = list(Ev_path.glob(Ev_lev_path_glob))
  Ev_ringdown_lev_paths = list(Ev_path.glob("Lev?_Ringdown/"+Ev_lev_path_glob))
  levs = set()
  for paths in Ev_lev_paths:
    levs.add(str(paths.stem)[3])

  if has_previous_levs and len(levs)>1:
    raise Exception(f"{previous_levs_set=} is given, use Ev_lev_path_glob to select one specific lev in the {Ev_path=}")
  
  if has_previous_levs:
    # Find all the levs in the previous_lev_list
    # NOTE! Right now only one lev is supported in the previous_lev_list and that requires that Ev_lev_path_glob selects that Lev
    previous_levs_set = set()
    for path in previous_levs_list:
      previous_levs_set.add(str(path.stem)[3])

    if len(previous_levs_set)>1:
      # User gave multiple levs in the preivous lev list, this is not supported directly!
      raise Exception(f"{previous_levs_list=} has more than one lev {previous_levs_set=} which is not supported!")
    
    if levs != previous_levs_set:
      raise Exception(f"Levs globbed from {Ev_path=}:\n {levs=} \n are different from the levs given in the {previous_levs_set=}:\n {previous_levs_set=}")

  GW_data_dict['levs'] = list(levs)
  for lev in levs:
    # Add the lev paths in increasing time order
    GW_data_dict[f'Lev{lev}_dir_list'] = sort_paths_by_postfix(Ev_lev_paths)
    GW_data_dict[f'Lev{lev}_dir_list'] = GW_data_dict[f'Lev{lev}_dir_list']+sort_paths_by_postfix(Ev_ringdown_lev_paths)

    # Prepend the previous levs if given
    if has_previous_levs:
      GW_data_dict[f'Lev{lev}_dir_list'] = sort_paths_by_postfix(previous_levs_list)+GW_data_dict[f'Lev{lev}_dir_list']

    # Add the GW data for this lev
    GW_data_dict[f"Lev{lev}_GW_data"] = get_GW_data_for_levs(GW_data_dict[f'Lev{lev}_dir_list'])

    # Create folders to save the data
    for key in GW_data_dict[f"Lev{lev}_GW_data"]:
      if key == "bondi_radii_list":
        continue
      else:
        save_folder = save_path/key
        save_folder.mkdir()
        joined_data_name = key+".h5" # CCE needs the radius to be bw R and .h5
        submit_joinH5_command(save_folder,GW_data_dict[f"Lev{lev}_GW_data"][key],joined_data_name,write_scripts_only)

        # For bondi cce data create the yaml files and submission scripts for the cce as well
        if "Bondi" in key:
          cce_input_file_path = make_config_file(save_folder/joined_data_name)
          make_submit_file(save_folder,cce_input_file_path,CCE_Executable_path,write_scripts_only)

    serialize_and_dump_dict(GW_data_dict,save_path/"GW_data_dict.json")
  return GW_data_dict

In [ ]:
Ev_path = Path(f"/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian")
Ev_path = Path(f"/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra_200")
previous_levs_list = [
        Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_AA"),
        Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_AB"),
        Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_AC"),
        Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/Lev5_AD"),
      ]
CCE_executable = Path("/groups/sxs/hchaudha/spec_runs/CCE_stuff/CceExecutables/CharacteristicExtract")
asd = write_GW_data_dict(Ev_path,Ev_lev_path_glob="Lev5_?[A-Z]",previous_levs_list=previous_levs_list,write_scripts_only=True,CCE_Executable_path=CCE_executable)

In [ ]:
runs_list = [
  "Lev5_big_gaussian",
  "Lev5_big_gaussian_ah_tol10",
  "Lev5_big_gaussian_ah_tol100",
  "Lev5_big_gaussian_constra",
  "Lev5_big_gaussian_constra_200",
]

In [ ]:
for run_name in runs_list:
  lev_paths_dict = {}
  Ev_path = Path(f"/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/{run_name}")
  save_path = Ev_path/"GW_data"

  lev_paths_dict[run_name] = {
    "Ev_path":Ev_path,
    "save_path":save_path,
  }

  lev_paths = list(Ev_path.glob("Lev?_??"))
  levs = set()
  for paths in lev_paths:
    levs.add(str(paths.stem)[3])

  # save_path.mkdir(parents=True, exist_ok=False)

  for lev in levs:
    lev_paths = []
    lev_paths = lev_paths + list(Ev_path.glob(f"Lev{lev}_??"))
    lev_paths = lev_paths + list(Ev_path.glob(f"Lev{lev}_Ringdown/Lev{lev}_??"))
    lev_paths.sort()
    lev_paths_dict[run_name][f"Lev{lev}"] = path_list_to_str_list(lev_paths)

  # with (save_path/"paths.json").open('w') as f:
  #   json.dump(path_dict_to_str_dict(lev_paths_dict),f,indent=2)

  with (save_path/"paths.json").open('w') as f:
    json.dump(path_dict_to_str_dict(lev_paths_dict),f,indent=2)

## Write JoinH5 and write files

In [ ]:
lev_paths_dict

In [ ]:
for lev in [0,1,2]:
  lev_paths = []
  lev_paths = lev_paths + list(Path(f"/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev").glob(f"Lev{lev}_??"))
  lev_paths = lev_paths + list(Path(f"/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev").glob(f"Lev{lev}_Ringdown/Lev{lev}_??"))
  lev_paths.sort()
  path_list_to_str_list(lev_paths)

In [ ]:
GW_data = dict()
GW_data["bondi"] = list(Ev_path.glob("Lev?_??/Run/GW2/Bondi*.h5"))
GW_data["PhiMinus_FiniteRadii_CodeUnits"] = list(Ev_path.glob("Lev?_??/Run/GW2/PhiMinus_FiniteRadii_CodeUnits.h5"))
GW_data["PhiPlus_FiniteRadii_CodeUnits"] = list(Ev_path.glob("Lev?_??/Run/GW2/PhiPlus_FiniteRadii_CodeUnits.h5"))
GW_data["RawStrahlkorperIntegrals"] = list(Ev_path.glob("Lev?_??/Run/GW2/RawStrahlkorperIntegrals.h5"))
GW_data["rh_FiniteRadii_CodeUnits"] = list(Ev_path.glob("Lev?_??/Run/GW2/rh_FiniteRadii_CodeUnits.h5"))
GW_data["rPsi4_FiniteRadii_CodeUnits"] = list(Ev_path.glob("Lev?_??/Run/GW2/rPsi4_FiniteRadii_CodeUnits.h5"))

for key in GW_data:
  GW_data[key].sort()

In [ ]:
GW_data